# 6.3 모델 구현, 학습 및 결과 확인 

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
n_hidden = 35
lr = 0.01
epochs = 1000

string = "hello pytorch. how long can a rnn cell remember?" # show us your limit!
chars = "abcdefghijklmnopqrstuvwxyz ?!.,:;01"
char_list = [i for i in chars]
n_letters = len(char_list)

In [3]:
def string_to_onehot(string):
    start = np.zeros(shape=len(char_list), dtype=int)
    end = np.zeros(shape=len(char_list), dtype=int)
    start[-2] = 1
    end[-1] = 1
    for i in string:
        idx = char_list.index(i)
        zero = np.zeros(shape= n_letters, dtype=int)
        zero[idx]=1
        start = np.vstack([start, zero])
    output = np.vstack([start, end])
    return output

In [4]:
string_to_onehot('abc')

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])

In [5]:
def onehot_to_word(onehot_1):
    onehot = torch.Tensor.numpy(onehot_1)
    return char_list[onehot.argmax()]

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size

        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.i2o = nn.Linear(hidden_size, output_size)
        self.act_fn = nn.Tanh()
    def forward(self, input, hidden):
        hidden = self.act_fn(self.i2h(input)+self.h2h(hidden))
        output = self.i2o(hidden)
        return output, hidden
    
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)
    

rnn = RNN(n_letters, n_hidden, n_letters)

In [7]:
loss_func = nn.MSELoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [8]:
one_hot = torch.from_numpy(string_to_onehot(string)).type_as(torch.FloatTensor())

for i in range(epochs):
    rnn.zero_grad()
    total_loss = 0
    hidden = rnn.init_hidden()

    for j in range(one_hot.size()[0]-1):
        input_ = one_hot[j:j+1,:]
        target = one_hot[j+1]

        output, hidden = rnn.forward(input_, hidden)
        loss = loss_func(output.view(-1), target.view(-1))
        total_loss += loss
        input_ = output

    total_loss.backward()
    optimizer.step()

    if i % 10 == 0:
        print(total_loss)

tensor(2.3692, grad_fn=<AddBackward0>)
tensor(0.9361, grad_fn=<AddBackward0>)
tensor(0.6329, grad_fn=<AddBackward0>)
tensor(0.4139, grad_fn=<AddBackward0>)
tensor(0.2877, grad_fn=<AddBackward0>)
tensor(0.2031, grad_fn=<AddBackward0>)
tensor(0.1601, grad_fn=<AddBackward0>)
tensor(0.1226, grad_fn=<AddBackward0>)
tensor(0.1092, grad_fn=<AddBackward0>)
tensor(0.0911, grad_fn=<AddBackward0>)
tensor(0.0800, grad_fn=<AddBackward0>)
tensor(0.0784, grad_fn=<AddBackward0>)
tensor(0.0671, grad_fn=<AddBackward0>)
tensor(0.0651, grad_fn=<AddBackward0>)
tensor(0.0552, grad_fn=<AddBackward0>)
tensor(0.0499, grad_fn=<AddBackward0>)
tensor(0.0441, grad_fn=<AddBackward0>)
tensor(0.0449, grad_fn=<AddBackward0>)
tensor(0.0420, grad_fn=<AddBackward0>)
tensor(0.0330, grad_fn=<AddBackward0>)
tensor(0.0287, grad_fn=<AddBackward0>)
tensor(0.0338, grad_fn=<AddBackward0>)
tensor(0.0246, grad_fn=<AddBackward0>)
tensor(0.0239, grad_fn=<AddBackward0>)
tensor(0.0220, grad_fn=<AddBackward0>)
tensor(0.0193, grad_fn=<A

In [9]:
start = torch.zeros(1, len(char_list))
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        output_string += onehot_to_word(output.data)
        input_ = output


print(output_string)

hello gmcelcerceellcelwreleel.melcernmcelcellcel


In [10]:
start = torch.zeros(1, len(char_list))
start[:,-2] = 1

with torch.no_grad():
    hidden = rnn.init_hidden()
    input_ = start
    output_string = ""
    for i in range(len(string)):
        output, hidden = rnn.forward(input_, hidden)
        print(onehot_to_word(output.data))
        input_ = output

h
e
l
l
o
 
g
m
c
e
l
c
e
r
c
e
e
l
l
c
e
l
w
r
e
l
e
e
l
.
m
e
l
c
e
r
n
m
c
e
l
c
e
l
l
c
e
l


### 임베딩

In [12]:
# !pip install unidecode

In [13]:
import unidecode
import string
import random
import re
import time, math

In [14]:
all_characters = string.printable
n_characters = len(all_characters)
print(all_characters)
print('num_chars = ', n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100
